<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3: Webscaper Function/Pushshift API

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests
import urllib
import time
import json

### Pushshift API Function

In [2]:
def get_data(object_type, username='',subreddit='', search_query='', max_time=None, min_time=1577860720):

    if max_time is None:
        max_time = int(time.time())

    filter_string = urllib.parse.urlencode(
        {k: v for k, v in zip(
            ['title','subreddit','created_utc','selftext'],
            [username,subreddit, search_query]) if v != ""})

    url_format = "https://api.pushshift.io/reddit/search/{}/?size=100&sort=desc&{}&before={}"

    before = max_time
    df = pd.DataFrame()
    
    while before > min_time:
        url = url_format.format(object_type, filter_string, before)
        resp = requests.get(url)

        
        dfi = pd.json_normalize(json.loads(resp.text)['data'])
        
        if object_type == 'comment':
            dfi = dfi.rename(columns={'created_utc': 'date', 'body': 'comment'})
            df = pd.concat([df, dfi[['id','date', 'comment', 'score','subreddit']]])
        elif object_type == 'submission':
            dfi = dfi.rename(columns={'created_utc': 'date', 'selftext': 'post'})
            dfi = dfi[dfi['post'].ne('')]
            df = pd.concat([df, dfi[['id','date', 'post', 'score','subreddit']]])

        before = dfi['date'].min()

        # Keep proper pace in reddit
        time.sleep(15)
        
    return df

#https://stackoverflow.com/questions/66783488/code-efficiency-performance-improvement-in-pushshift-reddit-web-scraping-loop

### Subreddit(Coffee)

In [ ]:
# df_commentsername=""
# subreddit = "coffee"
# search_query = ""

# df_comments = get_data(
#     object_type='submission',
#     username=username,
#     subreddit=subreddit,
#     search_query=search_query)
# # january 1st 2021 1609483120


In [ ]:
# df_comments.to_csv('../data/coffee.csv', index=False)
# should have used index=False

### Subreddit(Tea)

In [ ]:
# username=""
# subreddit="tea"
# search_query=""

# tea_posts=get_data(object_type='submission',
#                   username=username,
#                   subreddit=subreddit,
#                   search_query=search_query)
# #posts since january 1,2020 to get about as much data as the coffee subreddit.(UTC=1577860720)
# tea_posts.info()
# tea_posts.to_csv('../data/tea.csv', index=False)